In [ ]:
#hidden cell to be executed BEFORE the presentation
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import dftpy
from dftpy.ions import Ions
from dftpy.field import DirectField
from dftpy.grid import DirectGrid
from dftpy.functional import LocalPseudo, Functional, TotalFunctional
from dftpy.formats import io
from dftpy.math_utils import ecut2nr
from dftpy.time_data import TimeData
from dftpy.optimization import Optimization
from dftpy.mpi import sprint
from IPython.lib.display import YouTubeVideo
from IPython.display import IFrame
from ase.visualize import view
PP_list = {'Al': 'Al_lda.oe01.recpot'}
#import fortecubeview

<center>
    <h1>KS DFT, nonlocal PPs and simulations with the QE-powered <b>QEpy</b></h1>
<center>
<br>
<table>
  <tr>
      <td><p><h1>Team Rutgers</h1></p><p><h2>Rutgers University-Newark</h2></p></td>
      <td><img src="figures/logos/run.png" width=300 height=300 /></td>
  </tr>
  <tr>
    <td></td>
    <td> http://prg.rutgers.edu</td>
  </tr>
</table>
<br>
<br>
<br>

#### Quantum Multiscale School -- Boise, Idaho -- June 24-28, 2024


# Goals of this lecture + hands-on session
- More in depth analysis of KS-DFT
- Nonlocal part of pseudopotentials
- The Self-consistent Field method
    - Handling occupations (smearing)
    - Handling density mixing (convergence accelerators)
- Account of periodicity (Bloch theorem)
    - Handling k-point sampling

# Solvers
<br>
<br>
<table border="1" style="width:100%; text-align:center;">
    <tr>
        <th></th>
        <th style="text-align:center;border-right: 2px solid red;padding: 40px;">OF-DFT</th>
        <th style="text-align:center;">KS-DFT</th>
    </tr>
    <tr>
        <th style="text-align:center;padding: 40px;">Direct Minimization</th>
        <td style="text-align:center; border-right: 2px solid red; background-color: lightgreen;">
            \( n_0(\mathbf{r})=\arg\underset{n}\min\big\{ \mathcal{L}_{OF}[n]\big\} \)
        </td>
        <td style="text-align:center;">
            \( \{\phi_i^0\} = \arg\underset{\{\phi_i\}}\min\big\{ \mathcal{L}_{KS}[\{\phi_i\}]\big\} \)
        </td>
    </tr>
    <tr>
        <th style="text-align:center;padding: 40px;">SCF</th>
        <td style="text-align:center;border-right: 2px solid red; ">
            \( -\frac{1}{2}\nabla^2 \sqrt{n(r)} + v_B(r)\sqrt{n(r)} = \mu\sqrt{n(r)}\)
        </td>
        <td style="text-align:center;background-color: lightgreen;">
            \( -\frac{1}{2}\nabla^2 \phi_i(r) + v_s(r)\phi_i(r) = \varepsilon_i\phi_i(r)\)
        </td>
    </tr>
</table>


# Basis sets (discretization)

To be able to run simulations, we need to discretize the space in which the KS and OF wavefunctions / densities live. We will consider plane waves (PW) and Gaussian-type orbitals (GTOs). The general idea is the following:
$$
\phi_i(r) = \sum_\mu^M c_\mu \chi_\mu(r) \text{, where } \chi_\mu \text{ are basis functions}
$$

<table border="1" style="width:100%; text-align:center;">
    <tr>
        <th></th>
        <th style="text-align:center;border-right: 2px solid red;padding: 40px;">GTOs</th>
        <th style="text-align:center;">PW</th>
    </tr>
    <tr>
        <th style="text-align:center;padding: 40px;">Definition</th>
        <td style="text-align:center; border-right: 2px solid red;">
            \( \chi_\mu(r) = x^{i_\mu}y^{j_\mu}z^{k_\mu} e^{-\frac{|r-R_\mu|^2}{2\sigma_\mu^2}} \)
        </td>
        <td style="text-align:center;">
            \( \chi_\mu(r) = \frac{1}{\sqrt{\Omega}} e^{i G_\mu \cdot r},\,\,G_\mu = \left(\frac{2\pi\, i_\mu}{a},\frac{2\pi\, j_\mu}{b},\frac{2\pi\, k_\mu}{c}\right)\)
        </td>
    </tr>
    <tr>
        <th style="text-align:center;padding: 40px;">Location</th>
        <td style="text-align:center;border-right: 2px solid red;">
            \( R_\mu \in \text{centers of atoms}\)
        </td>
        <td style="text-align:center;">
            Spread out throughout the simulation cell
        </td>
    </tr>
    <tr>
        <th style="text-align:center;padding: 40px;">Number of functions</th>
        <td style="text-align:center;border-right: 2px solid red;">
            \( \simeq 10 \) per atom
        </td>
        <td style="text-align:center;">
            \( \propto \Omega = a \cdot b \cdot c \), generally \( 10^4 \) to \( 10^6 \)
        </td>
    </tr>
    <tr>
        <th style="text-align:center;padding: 40px;">Handling eN potential</th>
        <td style="text-align:center;border-right: 2px solid red;">
            <center><img src="./figures/science/fullpot.png" alt="GTOs image" width=400 ></center>
        </td>
        <td style="text-align:center;">
            <center><img src="./figures/science/pseudopot.png" alt="PW image" width=400 ></center>
        </td>
    </tr>
</table>


# Valence and Core orbitals
<br>
<br>
<center><img src="./figures/science/pseudo_core.png" alt="econf" width=800 >

<h1> Local and <span style="color: red;">nonlocal</span> parts of the pseudo potential</h1>

<h2>What is the non-local pseudo-potential?</h2>

- NL-PPs make sure valence orbitals are orthogonal to core orbitals
- NL-PPs are present in all KS-DFT PPs
- OF-DFT cannot use NL-PPs <span style="color: red;">because there are no orbitals!</span>

The idea is: apply Gram-Schmidt orthogonalization to the set of valence, $|\phi\rangle$, and core, $|\text{core}\rangle$, electrons:
$$
|\phi\rangle \to |\phi\rangle - |\text{core}\rangle\langle\text{core}|\phi\rangle = \bigg[ 1 - |\text{core}\rangle\langle\text{core}| \bigg] |\phi\rangle 
$$

<center>
<div class="alert alert-warning">$\hat{P}_{nl}=\bigg[ 1 - |\text{core}\rangle\langle\text{core}| \bigg]$ is the <b>non-local pseudopotential</b></div>
</center>

<center>
    <div class="alert alert-success">$$v_{eN}(r) \to v_{PP}^{loc}(r) + \hat v_{PP}^{non-loc}$$</div>
</center>

# The Self-Consistent Field Method

The KS equations $-\frac{1}{2}\nabla^2 \phi_i(r) + v_s(r)\phi_i(r) = \varepsilon_i\phi_i(r)$ feature $v_s(r)$ which depends on the density:
<br>
<br>
$$
v_s[n](r) = \frac{\delta E_{H}[n]}{\delta n(r)} + \frac{\delta E_{xc}[n]}{\delta n(r)} + v_{eN}(r)
$$

But the density depends on the KS orbitals, $\{\phi_i(r)\}$:
<br>
<br>
$$
n(r) = \sum_i n_i |\phi_i(r)|^2\,\, n_i \text{ are the occupation numbers.}
$$

<center>
<div class="alert alert-danger">Can the problem be solved?</div>
</center>

<center>
<img src="./figures/science/SCF_scheme.png" width=1200 \>
</center>

<center>
<div class="alert alert-success">Most times it converges :)</div>
</center>

# Setting up QEpy and QE's input file for KS-DFT

In [ ]:
from qepy.driver import Driver
from qepy.io import QEInput

In [ ]:
IFrame(src="https://www.quantum-espresso.org/Doc/INPUT_PW.html",width=1250, height=600)

### QEpy's input file is a dictionary containing QE's input keywords

In [ ]:
from dftpy.formats import download_files
additional_files = {'Al.pbe-nl-kjpaw_psl.1.0.0.UPF' : 'https://pseudopotentials.quantum-espresso.org/upf_files/Al.pbe-nl-kjpaw_psl.1.0.0.UPF'}
download_files(additional_files)

In [ ]:
qe_options = {
    '&control': {
        'calculation': "'scf'",
        'pseudo_dir': "'./'",
    },
    '&system': {
        'ibrav' : 0,
        'degauss': 0.005,
        'ecutwfc': 30,
        'occupations': "'smearing'"
    },
    'atomic_species': ['Al  26.98 Al.pbe-nl-kjpaw_psl.1.0.0.UPF'],
    'k_points gamma': [],
}

In [ ]:
#!cat Al.pbe-nl-kjpaw_psl.1.0.0.UPF

In [ ]:
options = {
    '&electrons': {
        'mixing_beta': 0.5},
    'cell_parameters angstrom':[
        '0.     2.025  2.025',
        '2.025  0.     2.025',
        '2.025  2.025  0.   '],
    'atomic_positions crystal': ['Al    0.0  0.0  0.0'],
    'k_points automatic': ['6 6 6 1 1 1'],
}

# Wait... what?

## $\bullet$ smearing?
## $\bullet$ mixing_beta?
## $\bullet$ k-points?
## $\bullet$ DFT functionals?

Let's first run QEpy and solve the SCF for our simple Al system:

In [ ]:
qe_options = QEInput.update_options(options, qe_options=qe_options)

In [ ]:
driver = Driver(qe_options=qe_options, logfile=True)
%timeit -n1 -r1 driver.scf() 

# Smearing

$$
n(r) = \sum_i n_i |\phi_i(r)|^2,\qquad \{n_i\} \text{ are the occupation numbers.}
$$

Consider the electronic DOS of Aluminum:

<center><img src="figures/science/DOS_example.png" width=1000 /></center>

<center>
<div class="alert alert-danger">Not possible to use integer occupations. The SCF would never converge!</div>
</center>

# Types of smearing

<b>Gaussian</b>: $$n_i = \frac{1}{2}\text{Erfc}\left[\frac{|\varepsilon_i-\mu|}{\text{degauss}}\right]$$

<b>Fermi-Dirac</b>: $$n_i = \left[ e^{\frac{|\varepsilon_i-\mu|}{\text{degauss}}}+1\right]^{-1}$$

<b>Others</b>: To be used in specific situations (semicondictors, or others), see [m-p](https://journals.aps.org/prb/abstract/10.1103/PhysRevB.40.3616) and [m-v](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.82.3296).

# Density mixing

Broyden, Pulay (DIIS), ..., are SCF convergence accelerators. They all share the same logic:

1) Consider a trial of densities $\{ n_i(r) \}$ which are outputs of the $i$-th SCF cycle.

2) The converged (SCF) density is given as linear combination of the trials, $n = \sum_i c_i n_i$.

3) The "best" coefficients at each SCF step are found minimizing the error vector with respect to the coefficients. The error in QE is given by $R = \sum_i c_i R_i$ which is given in terms of the density residue, $\delta n_i = n_i - n_{i-1}$, as $R_i = \int \frac{\delta n_i(r)\delta n_i(r')}{|r-r'|}drdr'$.

4) A new density is generated with the new coefficients and the trial vectors to be reinserted in the SCF loop.

# Example with QEpy

Let's run QEpy with default QE's Broyden mixing:

In [ ]:
driver=Driver(qe_options=qe_options, iterative = True, logfile='tmp.out') 
for i in range(60):
    driver.diagonalize()
    driver.mix()
    converged = driver.check_convergence()
    print ('Iter: ',i,' - Conv: ', driver.get_scf_error())
    if converged : break
driver.calc_energy()

<center>
    <div class="alert alert-success">Remember: 3 iterations.</div>
</center>

Let's run QEpy with a simple linear mixing, $n \to \alpha \, n_\text{prev} + (1- \alpha) \, n$

In [ ]:
from dftpy.functional import Hartree 
driver=Driver(qe_options=qe_options, iterative = True, logfile='tmp.1.out') 
rho = driver.get_density().copy()
alpha = 0.7
for i in range(20):
    driver.diagonalize()
    #
    rho_new = driver.get_density().copy()
    drho = driver.data2field(rho_new-rho)
    error = Hartree.compute(drho).energy
    nc = np.abs(drho).sum()*driver.get_volume()/drho.size
    print ('Iter: ',i,' - Conv: ', error, 'dN:', nc)
    if error < 1e-8:
        driver.end_scf()
        break
    #
    rho = (1-alpha)*rho + alpha * rho_new
    driver.set_density(rho)
driver.calc_energy()

<center>
    <div class="alert alert-danger">5 iterations...</div>
</center>

<center>
    <div class="alert alert-success">Density mixing <b>does</b> improve convergence.</div>
</center>

# Bloch theorem and k-point sampling

1) Solids have periodic potentials:
$$
v_{eN}(r+n\hat a) = v_{eN}(r),\,\, \text{where } \hat a \text{ is a lattice vector.}
$$

<center><img src="figures/science/periodic_pot.png" width=1300 /></center>

2) Bloch theorem states that the group of translations comes with 3 quantum numbers $\vec k = (k_a, k_b, k_c)$ representing translations along the 3 lattice vectors. The wavefunctions will be labelled by $k$ (dropped $\vec k$ for a lighter notation):
$$
\phi_i \to \phi_{ik} = e^{ik\cdot r} u_{ik}(r), \,\, k\in \text{First Brillouin Zone (FBZ)}
$$

FBZ: the set of $k$ vectors generating a phase $e^{ik\cdot r}$ that is periodic by no less than a full cell length ($k_{a/b/c} \leq 0.5$).

# K-point sampling example with QEpy

1) First, let's see how many orbitals (bands) we have.

In [ ]:
print(f"Number of orbitals (bands) considered by QE: {driver.get_number_of_bands()}") 

2) Let's take a look at the irreducible k-points considered by QE.

In [ ]:
driver.get_ibz_k_points()

3) Check that the total number of electrons is the sum of all occupations (times the k-point weight) over all k-points and all bands:

In [ ]:
N=[]
for i in range(driver.get_number_of_k_points()):
    N.append(driver.get_occupation_numbers(kpt=i).sum())
N=np.asarray(N)
N.sum()

# DFT functionals

<b>Pure functionals</b>, dependent on $n(r)$, $|\nabla n(r)|$, $\nabla^2 n(r)$ and or $\tau_{s}(r) = \frac{1}{2}\sum_i n_i |\nabla\phi_i(r)|^2$:
- LDA
- PBE (GGA)
- SCAN (meta-GGA)

<b>Hybrid functionals</b>, dependent on pure functionals and on HF exchange, $E_x^{HF}[\{\phi\}]$.
- PBE0
- B3LYP
- HSE

<b>Hubbard functionals</b> which are inspired by GW and will be discussed later (Iurii Timorov).
- GGA+U, GGA+U+V

# The self-interaction error in DFT

There are constraints that are hard to satisfy, chiefly among them:

- An electron should not interact with itself!

- $E_x[n] = - E_{H}[n]$ for 1 and 2 electron systems. Not for any other system.
    - LDA and PBE don't even try to satisfy this constraint
    - SCAN does by comparing $\tau_s(r)$ with $\tau_{vW}(r)$, but it needs to interpolate. The interpolation is tricky.

- You will often see the following picture from <a href="https://www.science.org/doi/abs/10.1126/science.1158722">this paper</a> which fully describes this issue

<center><img src="./figures/science/SIE.png" width = 1000 /></center>

# Ready for some hands-on work?

- KS-DFT with QEpy and QE
- Band structures and DOS
- Equation of state: equilibrium volume and energy, bulk modulus
- Convergence tests (PW cutoff, k-point sampling, smearing parameters)